In [ ]:
# Challenge 3 - 2025/05/29
# - Implement an LCEL chain with a memory that uses one of the memory classes we learned about.
# - The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
# - Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.
# - To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import ConversationChain
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    FewShotPromptTemplate,
    PromptTemplate,
)

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {"question": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"question": "The Godfather", "answer": "👨‍👨‍👦🔫🍝"},
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["question", "answer"],
        template="question: {question}\nanswer: {answer}",
    ),
    suffix="question: {movie}\nanswer:",
    input_variables=["movie"],
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI that responds to a user's question about the name of a movie with three emojis related to it.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"question": question},
        {"answer": result.content},
    )
    print(result.content)

In [2]:
invoke_chain("The Avengers")

🦸‍♂️💥🌎


In [3]:
invoke_chain("Harry potter")

⚡🧙‍♂️🏰


In [ ]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

conversation.predict(
    input="tell me what is the movie I asked first. Please just tell me the only name."
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='The human mentions The Avengers and Harry Potter to the AI, who responds with emojis representing superheroes and magic. The AI then provides a detailed description of "The Avengers," explaining the plot and characters in the film.'), HumanMessage(content='tell me what is the movie I asked first.'), AIMessage(content='The movie you asked first is "The Avengers." It is a superhero film based on the Marvel Comics superhero team of the same name. The movie features iconic characters such as Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye, who come together to save the world from a powerful threat. The film was directed by Joss Whedo

'The movie you asked first is "The Avengers."'